# **Dynamic Pricing for Urban Parking Lots**
Final Capstone Project for the Summer Analytics course by CA-IITG

In [1]:
!pip install pathway bokeh --quiet

In [2]:
#importing necessary libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from datetime import datetime

import pathway as pw
import panel as pn
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column

output_notebook()

In [3]:
#reading the dataset file
df = pd.read_csv('/content/dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,4/10/2016,7:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,4/10/2016,8:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,4/10/2016,8:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,4/10/2016,9:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,4/10/2016,9:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [4]:
#mixing date and time into a single timestamp
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='mixed', dayfirst=True)
df = df.sort_values(['SystemCodeNumber', 'timestamp']).reset_index(drop=True)
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,4/10/2016,7:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,4/10/2016,8:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,4/10/2016,8:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,4/10/2016,9:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,4/10/2016,9:59:00,2016-10-04 09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00,2016-12-19 14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00,2016-12-19 15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00,2016-12-19 15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00,2016-12-19 16:03:00


# **Exploratory Data Analysis:**

In [5]:
print(df['TrafficConditionNearby'].unique())
print(df['VehicleType'].unique())

['low' 'high' 'average']
['car' 'bike' 'truck' 'cycle']


In [6]:
#encoding traffic and vehicle values to numbers for pricing
traffic_map = {'low':0, 'average':1, 'high':2}
vehicle_map = {'cycle':1, 'bike':2, 'car':3, 'truck':4}

df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp,TrafficLevel,VehicleWeight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,4/10/2016,7:59:00,2016-10-04 07:59:00,0,3
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,4/10/2016,8:25:00,2016-10-04 08:25:00,0,3
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,4/10/2016,8:59:00,2016-10-04 08:59:00,0,3
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,4/10/2016,9:32:00,2016-10-04 09:32:00,0,3
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,4/10/2016,9:59:00,2016-10-04 09:59:00,0,2


# **MODEL 1: Linear Baseline**

In [7]:
BASE_PRICE = 10;

In [8]:
def baseline_model(df, alpha=1.0):
  df['Price'] = BASE_PRICE

  price_log = {}
  for lot, group in df.groupby('SystemCodeNumber'):
    prices = []
    prev_price = BASE_PRICE
    for _, row in group.iterrows():
      occupancy_ratio = row['Occupancy']/row['Capacity']
      new_price = prev_price + alpha*occupancy_ratio
      new_price = max(5, min(20, new_price))
      prices.append(new_price)
      prev_price = new_price
    price_log[lot] = prices

  all_prices = []
  for lot in df['SystemCodeNumber'].unique():
    all_prices.extend(price_log[lot])
  df['Price'] = all_prices
  return df

# MODEL 2: Demand based cost function

In [9]:
#parameters can be changed
def demand_function(row, alpha=1.5, beta=0.5, gamma=0.3, delta=2, epsilon=1.0):
  occupancy_ratio = row['Occupancy']/row['Capacity']
  demand = (alpha*occupancy_ratio + beta*row['QueueLength'] - gamma*row['TrafficLevel'] + delta*row['IsSpecialDay'] + epsilon*row['VehicleWeight'])
  return demand

In [10]:
def demand_based_model(df, lambda_val=0.3):
  df['RawDemand'] = df.apply(demand_function, axis=1)

  min_demand = df['RawDemand'].min()
  max_demand = df['RawDemand'].max()
  df['NormDemand'] = df['RawDemand'].apply(lambda x: (x-min_demand)/(max_demand-min_demand)) #scaling demand to be in the range [0,1]

  df['Price'] = BASE_PRICE * (1+lambda_val*df['NormDemand'])
  df['Price'] = df['Price'].clip(lower=5, upper=20)
  return df

# MODEL 3: Competitive pricing

In [11]:
from geopy.distance import geodesic #to find distance between latitude and longitudes

def get_distances(df):
  lot_coords = df.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first()
  dist_matrix = {}
  for lot1 in lot_coords.index:
    dist_matrix[lot1] = {}
    for lot2 in lot_coords.index:
      dist_matrix[lot1][lot2] = geodesic(tuple(lot_coords.loc[lot1]), tuple(lot_coords.loc[lot2])).meters

  return dist_matrix

In [19]:
DISTANCES = get_distances(df)

def adjust_price3(df, radius=800):

  dist_map = get_distances(df)

  #retrieving last price
  last_known = df.groupby('SystemCodeNumber')['Price'].transform('last')

  #for comparison
  latest_dict = df.groupby('SystemCodeNumber')['Price'].last().to_dict()

  adjusted_prices = []

  for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    own_price = row['Price']

    #getting competitor prices
    comp_prices = []
    if lot in dist_map:
      for other_lot, dist in dist_map[lot].items():
        if dist <= radius and other_lot in latest_dict:
          comp_prices.append(latest_dict[other_lot])

    avg_comp = np.mean(comp_prices) if comp_prices else own_price

    #adjusting for competitors
    if row['Occupancy'] >= row['Capacity'] and own_price > avg_comp:
      new_price = own_price - 0.5
    elif own_price < avg_comp:
      new_price = own_price + 0.3
    else:
      new_price = own_price

    adjusted_prices.append(np.clip(new_price, 5, 20))

  df = df.copy()
  df['Price'] = adjusted_prices

  return df

# Plotting results

In [13]:
def plot_prices(df, lots):
  plots = []
  for lot in lots:
    lot_df = df[df['SystemCodeNumber'] == lot]
    p = figure(title=f"Price over time: {lot}", x_axis_type='datetime', width=600, height=250)
    p.line(lot_df['timestamp'], lot_df['Price'], line_width=2, color="blue", legend_label="Price")
    #p.line(lot_df['timestamp'], lot_df['Occupancy'], line_color="red", line_dash="dashed", legend_label="Occupancy")
    p.legend.location = "top_left"
    plots.append(p)
  show(column(*plots))


In [21]:
#model 1
df_baseline = baseline_model(df.copy(), alpha=1.5)

#model 2
df_demand = demand_based_model(df.copy(), lambda_val=0.4)

#model 3
df_comp = adjust_price3(df_demand.copy(), radius=800)

#plotting data for few lots
sample_lots = df['SystemCodeNumber'].unique()[:3]

print("Baseline Model")
plot_prices(df_baseline, sample_lots)
print("Demand Model")
plot_prices(df_demand, sample_lots)
print("Competitive Model")
plot_prices(df_comp, sample_lots)

Baseline Model


Demand Model


Competitive Model


(Couldn't properly get the pathway live data stream to work properly...)